In [2]:
import urllib.request
from urllib.error import HTTPError
import requests
i = 0

links = []
while True:
    page = requests.get("https://fr.wikisource.org/w/index.php?title=Special:IndexPages&limit=5000&offset=" + str(i) + \
                        "&key=&order=")
    print(page.content)
    from lxml import html
    content = html.fromstring(page.content)
    links_found = 0
    for item in content.xpath("//div"):
        if "prp-indexpages-row" in item.classes:
            links_found += 1
            link = item.findall('.//a')[0].get('href')
            name = link.split('/')[::-1][0]
            links.append((name, "https://fr.wikisource.org/w/index.php?title=" + name + "&action=raw"))
    if links_found == 0:
        break
    i += 5000

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



b'<!DOCTYPE html>\n<html class="client-nojs" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Liste des livres - Wikisource</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"IndexPages","wgNamespaceNumber":-1,"wgPageName":"Sp\xc3\xa9cial:IndexPages","wgTitle":"IndexPages","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":!1,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":!0,"wgPageContentLanguage":"fr","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\\t.","\xc2\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janvier","f\xc3\xa9vrier","mars","avril","mai","juin","juillet","ao\xc3\xbbt","septembre","octobre","novembre","d\xc3\xa9cembre"],"wgMonthNamesShort":["","janv.","f\xc3\xa9v.","

In [18]:
links

[('Livre:Godefroy_-_Lexique.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Godefroy_-_Lexique.djvu&action=raw'),
 ('Livre:Jaur%C3%A8s_-_Histoire_socialiste,_IV.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Jaur%C3%A8s_-_Histoire_socialiste,_IV.djvu&action=raw'),
 ('Livre:Jaur%C3%A8s_-_Histoire_socialiste,_III.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Jaur%C3%A8s_-_Histoire_socialiste,_III.djvu&action=raw'),
 ('Livre:Dictionnaire_de_Tr%C3%A9voux,_1771,_I.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Dictionnaire_de_Tr%C3%A9voux,_1771,_I.djvu&action=raw'),
 ('Livre:Platon_-_%C5%92uvres,_trad._Cousin,_I_et_II.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Platon_-_%C5%92uvres,_trad._Cousin,_I_et_II.djvu&action=raw'),
 ('Livre:Jaur%C3%A8s_-_Histoire_socialiste,_I.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Jaur%C3%A8s_-_Histoire_socialiste,_I.djvu&action=raw'),
 ('Livre:Cervantes_-_L%E2%80%99Ing%C3%A9ni

In [2]:

from lxml import html
content = html.fromstring(page.content)
links = []
for item in content.xpath("//div"):
    if "prp-indexpages-row" in item.classes:
        link = item.findall('.//a')[0].get('href')
        name = link.split('/')[::-1][0]
        links.append((name, "https://fr.wikisource.org/w/index.php?title=" + name + "&action=raw"))

print(links[0])

IndexError: list index out of range

In [9]:
import re

page = requests.get(links[0][1])
print(page.content)
page.encoding = 'utf-8'
page_content = str(page.content).split('|', 1)[1]
headings = re.sub(r"(\|Remarks).*(\|Notes)", r'\1=\n\2', str(page.content)).split('|')
headings.remove(headings[0])
count = 0
s = ""
for heading in headings:
    prop, val = heading.split('=', 1)
    s += prop + "\t"
    print(prop)
print(s[:-1], file=open("bn-ws.tsv", "w+"))

b"{{:MediaWiki:Proofreadpage_index_template\n|Type=book\n|Titre=[[Lexique de l\xe2\x80\x99ancien fran\xc3\xa7ais]]\n|Sous_titre=\n|Volume=\n|Auteur=[[Auteur:Fr\xc3\xa9d\xc3\xa9ric Godefroy|Fr\xc3\xa9d\xc3\xa9ric Godefroy]]\n|Traducteur=\n|Editeur_scientifique=\n|Illustrateur=\n|Editeur=Librairie Honor\xc3\xa9 Champion\n|School=\n|Lieu=Paris\n|Annee=1890\n|Publication=\n|Bibliotheque=\n|Clef=Lexique de l\xe2\x80\x99ancien fran\xc3\xa7ais\n|BNF_ARK=\n|Source=djvu\n|Image=[[Image:Godefroy - Lexique (1).djvu|page=1|150px]]\n|Avancement=V\n|Pages=[[Page:Godefroy - Lexique (1).djvu/1|i]] - \n[[Page:Godefroy - Lexique (1).djvu/2|ii]] - \n[[Page:Godefroy - Lexique (1).djvu/3|iii]]\n\n[[Page:Godefroy - Lexique (1).djvu/4|1-g]]\n[[Page:Godefroy - Lexique (1).djvu/5|1-m]]\n[[Page:Godefroy - Lexique (1).djvu/6|1-d]] \n[[Page:Godefroy - Lexique (1).djvu/7|2-g]] \n[[Page:Godefroy - Lexique (1).djvu/8|2-m]] \n[[Page:Godefroy - Lexique (1).djvu/9|2-d]] \n[[Page:Godefroy - Lexique (1).djvu/10|3-g]] \n[

ValueError: not enough values to unpack (expected 2, got 1)

In [10]:
import codecs
from urllib.parse import unquote
import re
for y, l in links:
    page = requests.get(l)
    page.encoding = 'utf-8'
    page_content = str(page.content).split('|', 1)[1]
    headings = re.sub(r"(\|Remarks).*(\|Notes)", r'\1=\2', str(page.content))
    headings = re.sub(r"(\|Pages).*(\|Volumes)", r'\1=\2', headings)
    #print(headings)
    headings = headings.split('=')
    #print(headings)
    headings.remove(headings[0])
    s=unquote(y) + "\t" + "https://fr.wikisource.org/wiki/" + unquote(y) + "\t"
    for heading in headings:
        try:
            val, prop = heading.split('|', 1)
        except Exception:
            val = heading
        val = val.replace("}}","")
        val = val.replace("\\n", "")
        s += val + "\t"
    #print(s)
    with codecs.open("fr-ws.tsv", "a+", "utf-8-sig") as temp:
        temp.write(bytes(s, 'utf-8').decode('unicode_escape').encode('latin-1').decode('utf8') + "\n")

In [13]:
data = {'url': 'https://bn.wikisource.org/wiki/নির্ঘণ্ট:শ্রীক্ষেত্র-তত্ত্ব-সুধা.pdf'}
r = requests.post(url="https://meta.wikimedia.org/w/api.php?action=shortenurl", data=data)
print(r.text)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>MediaWiki API result - Meta</title>
<script>document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"ApiHelp","wgNamespaceNumber":-1,"wgPageName":"Special:ApiHelp","wgTitle":"ApiHelp","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":!1,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRelevantPag